In [ ]:
import shutil
source = r"C:\\Users\\Aarush Raj\\OneDrive\\Desktop\\img2rec\\DEEP-CHEF-PROJECT\\arym\\csv\\links.csv"
target = r"C:\\Users\\Aarush Raj\\OneDrive\Desktop\\img2rec\\DEEP-CHEF-PROJECT\\aarush\\linksc.csv"

shutil.copyfile(source, target)

In [ ]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pandas as pd
import io
import requests
from PIL import Image
import urllib3
#335 to 668 incl
#train folder and name of rceipe folder to be swapped in path

def fetch_image_content_from_url(url,timeout):
    try:
        http = urllib3.PoolManager(timeout=urllib3.Timeout(connect=None, read=None, total=timeout))
        response = http.request('GET', url)
        return response.data
    except Exception as e:
        print("Exception occurred inside fetch_image_content_from_url\n",e)
        
        
def download_image(download_path,url,filename):
    try:
        image_content = fetch_image_content_from_url(url,5)
        #requests.get(url,timeout=10)
        """if(img_content.status_code==200):
            image_content=img_content.content"""
        image_bytes = io.BytesIO(image_content)
        file_path = os.path.join(download_path,filename)
        image = Image.open(image_bytes)
        _image=image.convert("RGB")
        with open(file_path,"wb") as f:
            _image.save(f,"JPEG")
    except Exception as e:
        print("Exception is ", e)
        
        
def download_images(download_folder_path,urls,recipe,recipe_index,train_imgs_count):
    try:
        recipe_folder_name = str(recipe_index) + "_" + recipe
        train_folder_path = os.path.join(download_folder_path,"train",recipe_folder_name)
        test_folder_path = os.path.join(download_folder_path,"test",recipe_folder_name)

        if not os.path.exists(train_folder_path):
            os.makedirs(train_folder_path)
        if not os.path.exists(test_folder_path):
            os.makedirs(test_folder_path)
            
        img_number = 1
        for url in urls:
            if img_number <= train_imgs_count:
                download_image(train_folder_path,url,str(img_number)+"_"+recipe+".jpg")
            else:
                download_image(test_folder_path,url,str(img_number)+"_"+recipe+".jpg")
            img_number+=1
    except Exception as e:
        print("Exception is ", e)
        #log_urls(image_urls,recipe_index_csv,recipe,"dwd_logs.txt",no_of_img_to_train)
        
def log_urls(image_urls,recipe_index_csv,recipe,path_to_download_logs,train_imgs_count,ignore_msgs=False):
    try:
        with open(path_to_download_logs,"a") as f:
            f.write(str(recipe_index_csv)+"->"+ recipe +"\n")
            f.write("$$$$\n") # mark the start delimiter
            logged_urls = 0
            for url in image_urls:
                if logged_urls < train_imgs_count:
                    f.write("train:>"+url+"\n")
                else:
                    f.write("test:>" + url+"\n")
                logged_urls+=1
            f.write("$$$$\n")
            f.write("\n") # one empty line between end delimiter and new recipe
            f.flush()
        
    except Exception as e:
        if not ignore_msgs:
            print("Exception occurred while logging urls\n",e)
            
                    
# start web browser browser = webdriver.Chrome()
# change to joinpath
PATH="C:\\Users\\Aarush Raj\\OneDrive\\Desktop\\img2rec\\venv\\chromedriver.exe"
chrome_service=webdriver.ChromeService(executable_path=PATH)
chrome_options=Options()
chrome_options.add_experimental_option("detach",True)


try:
    # wait for search results to load
    start = 334
    row_count= 334
    big_delay = 7
    small_delay = 3
    max_imgs = 10
    extra_imgs = 5
    batch_size = 10
    no_of_img_to_train=8


    df = pd.read_csv("linksc.csv",skiprows= start,nrows=row_count,names=['name','link'])
    d_log=open("dwd_logs.txt","a")
    
    # download images
    for i in range(len(df['name'])):
        recipe=df.loc[i,'name']
        recipe_index_csv=i+start
        
        #d_log.write(str(recipe_index_csv)+"->"+ recipe +"\n")
        d_log.flush()
        
        driver=webdriver.Chrome(service=chrome_service,options=chrome_options)
        driver.maximize_window()
        driver.get('https://images.google.com/')
        time.sleep(7)
        search_box = driver.find_element(By.NAME,"q")
        search_box.send_keys(recipe)
        search_box.send_keys(Keys.ENTER)
        time.sleep(10)
    
        image_urls=list()
        # click on the first image
        thumbnails = driver.find_elements(By.CLASS_NAME,"mNsIhb")
        """while len(thumbnails) < (max_imgs + extra_imgs): # checking condition before scrolling to minimize HTTP Requests
            print("Less images obtained")
            time.sleep(small_delay)
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            time.sleep(small_delay)
            thumbnails = driver.find_elements(By.CLASS_NAME,"mNsIhb") 
        """    
        x=len(image_urls)
        count=0
        for thumbnail in thumbnails:
            if(count==max_imgs):
                break
            
            try:
                thumbnail.click()
                # wait for image to load
                time.sleep(5)
            except:
                continue
            #print(thumbnail)
            pop_up_window = driver.find_elements(By.CLASS_NAME,"jlTjKd")  
            for pop_up_elements in pop_up_window:
                if pop_up_elements.tag_name == 'a':
                    # print("<a> tag obtained")

                    # get all the img tags under <a>
                    img_tags = pop_up_elements.find_elements(By.TAG_NAME,"img")

                    # print("img tags obtained")
                    for img in img_tags:
                        class_name=img.get_attribute("class")
                        if "iPVvYb" in class_name.strip():
                            image_url=img.get_attribute('src')
                            if(image_url not in image_urls):
                                image_urls.append(image_url)
                                print("Found ",count)
                                count+=1
                                
                                # f.write("Found " + str(success_count) + "\n")
                            else:
                                print("Found Duplicate")
        
        download_images("C:\\Users\\Aarush Raj\\OneDrive\\Desktop\\img2rec\\DEEP-CHEF-PROJECT\\downloaded_images",image_urls,recipe,recipe_index_csv,no_of_img_to_train)
                    
        log_urls(image_urls,recipe_index_csv,recipe,"dwd_logs.txt",no_of_img_to_train)        
                # go back to search results
        time.sleep(7)        
    # close web browser
    #print(image_urls)
    d_log.close()
except Exception as e:
    print("Exception-",e)
    driver.quit()    